In [52]:
import requests
import re
import time
import pandas as pd

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36"

MAIN_ENDPOINT = f"https://poe.ninja/api/data/0/getbuildoverview"

PARAMS = {"overview": "ssf-archnemesis", "type": "exp", "language": "en"}

inventory_IDs = ['BodyArmour', 'Flask', 'Offhand', 'Gloves', 'Weapon', 'Boots', 'Belt', 'Ring', 'Ring2', 'Helm', 'Amulet']

def try_get_builds(url, params):
    r = requests.get(url=url, params=params)
    try:
        return r.json()
    except:
        return {}

data = try_get_builds(MAIN_ENDPOINT, PARAMS)
skills = [data['activeSkills'][i]['name'] for i in range(len(data['activeSkills']))]

def get_user_ascendancy(user_idx: int, api_data: dict):
    asc_idx = api_data["classes"][user_idx]
    return api_data["classNames"][asc_idx]

def summarize_player(idx, data):
    classes = data['classes']
    names = data['names']
    accounts = data['accounts']
    levels = data['levels']
    name = names[idx]
    person_class = classes[idx]
    class_name = get_user_ascendancy(idx, data)
    account = accounts[idx]
    level = levels[idx]
    # return (name, person_class, class_name, account)
    return (account, name, level)

def get_skill_enjoyers(skills, data):
    enjoyers = []
    for skill_id, skill_map in data['activeSkillUse'].items():
        if int(skill_id) in skills:
            enjoyers = [data['names'][x] for x in skill_map]
    return enjoyers

In [53]:
def find_characters_by_skill(skill_idx: int, data: dict):
    pointer = data['activeSkillUse'][str(skill_idx)][0]
    for i in data['activeSkillUse'][str(skill_idx)][1:]:
        yield pointer
        pointer += i

def get_first_n_by_skill(n: int, skill_idx: int, data: dict):
    return [x for x, _ in zip(find_characters_by_skill(skill_idx, data), range(n))]
    
def check_ascendancy_skill_char(chars_by_skill, ascendancy):
    lst = []
    for char in chars_by_skill:
        if get_user_ascendancy(char, data) == ascendancy: 
            lst.append(char)
    return lst

def get_first_n_by_skill_ascendancy(n: int, skill_idx: int, data: dict, ascendancy: str):
    return [x for x, _ in zip(check_ascendancy_skill_char(find_characters_by_skill(skill_idx, data), ascendancy), range(n))]

def get_account_and_name(n, skill, data, ascendancy):
    skill_idx = skills.index(skill)
    skill_asc =  get_first_n_by_skill_ascendancy(n, skill_idx, data, ascendancy)
    lst = []
    for c in skill_asc:
        lst.append(summarize_player(c, data))
    return sorted(lst, key = lambda x: x[2], reverse = True)

In [54]:
session = requests.session()
session.headers.update({"User-Agent": USER_AGENT})

In [55]:
def try_get_items(account, char):
    endpoint = "https://www.pathofexile.com/character-window/get-items"

    params = {"accountName": account, "character": char}

    r = session.get(url=endpoint, params = params)
    try:
        return r.json()
    except:
        return {}

In [56]:
def get_item(item_data):
    item = {}
    properties = ['name', 'inventoryId', 'baseType', 'ilvl', 'implicitMods', 'explicitMods', 'craftedMods', 'enchantMods', 'flavourText']
    for prop in properties:
        try: 
            item[prop] = item_data[prop]
        except:
            continue
    return item

def get_items(item_data, inventory_IDs = inventory_IDs):
    items = []
    for item in item_data:
        if item['inventoryId'] in inventory_IDs:
            items.append(get_item(item))
    return items

def get_chars_item_data(char_info: list, sleep = 1): 
    char_item_data = {}
    for account, char in char_info:
        char_item_data[char] = get_items(try_get_items(account, char)['items'])
        time.sleep(sleep)
    return char_item_data

def remove_numvalues_from_mod(mod):
        return re.sub(r'\d+', 'X', mod)
    
def tally_item_mods(char_item_data, implicits = False):
    # ex. {Weapon - Claw: {Adds X to X Fire Damage: 1, Adds X to X Cold Damage: 2}}
    # explicit/crafted mods only
    item_tally = {}
    for char in char_item_data.keys():
        for item in char_item_data[char]:
            mods = []
            modlists = ['explicitMods', 'craftedMods']
            item_type = item['inventoryId']

            if implicits: 
                modlists = ['implicitMods']
                eldritch_basetypes = ['Helm', 'BodyArmour', 'Gloves', 'Boots']
                if item_type not in eldritch_basetypes: 
                    continue

            for modlist in modlists: 
                if modlist in item:
                    for mod_data in item[modlist]: 
                        mods.append(remove_numvalues_from_mod(mod_data))    
            if item_type == "Ring2":
                item_type = "Ring"
            if item_type not in item_tally:
                mod_tally = {}
                for mod in mods:
                    mod_tally[mod] = 1
                item_tally[item_type] = mod_tally
            else:
                for mod in mods: 
                    if mod not in item_tally[item_type]:
                        item_tally[item_type][mod] = 1
                    else:
                        item_tally[item_type][mod] += 1
    return item_tally

def get_flask_types(char_item_data, enchant = False):
    # todo: enchants, unique -> flavor text
    flask_tally = {}
    for char in char_item_data.keys():
        for item in char_item_data[char]:
            item_type = item['inventoryId']
            if item_type == 'Flask': 
                flask_type = item['baseType']
                if 'flavourText' in item:
                    flask_type = item['name']
                if enchant:
                    flask_enchant = "No enchant"
                    if 'enchantMods' in item:
                        flask_enchant = item['enchantMods'][0]
                    #print(type(flask_type), type(flask_enchant), print(flask_enchant))
                    flask_enchant_combo = flask_type + " - " + flask_enchant
                    if flask_enchant_combo not in flask_tally:
                        flask_tally[flask_enchant_combo] = 1
                    else: 
                        flask_tally[flask_enchant_combo] += 1
                else:
                    if flask_type not in flask_tally:
                        flask_tally[flask_type] = 1
                    else: 
                        flask_tally[flask_type] += 1
    return flask_tally

def tally_weapon_enchant(char_item_data): 
    enchant_tally = {}
    for char in char_item_data.keys():
        for item in char_item_data[char]:
            item_type = item['inventoryId']
            if item_type == 'Weapon': 
                weapon_type = item['baseType']
                weapon_enchant = "No enchant"
                if 'enchantMods' in item:
                    weapon_enchant = item['enchantMods'][1]
                weapon_enchant_combo = weapon_type + " - " + weapon_enchant
                if weapon_enchant_combo not in enchant_tally:
                    enchant_tally[weapon_enchant_combo] = 1
                else: 
                    enchant_tally[weapon_enchant_combo] += 1
    return enchant_tally

In [58]:
myaccounts = [("iAwesom", "PraygeHelix"), ("iAwesom", "PraygeChump")]
char_item_data = get_chars_item_data(myaccounts)

In [59]:
get_flask_types(char_item_data, True)

{'Bottled Faith - No enchant': 1,
 'Granite Flask - Used when Charges reach full': 2,
 'Divine Life Flask - No enchant': 2,
 'Diamond Flask - Used when Charges reach full': 2,
 'Quartz Flask - Used when Charges reach full': 2,
 'Jade Flask - Used when Charges reach full': 1}

In [60]:
accounts = get_account_and_name(100, "Cyclone", data, "Slayer")
accounts[0:5]

[('black943', 'feefefefefeee', 100),
 ('Trashgarbag', 'swiftbonerstrike', 98),
 ('Righteous0ne', 'ShockwaveAbuser', 98),
 ('Lagartootemido', 'PeripatheticoNem', 98),
 ('dy391', 'Trashdude', 97)]

In [64]:
tally_item_mods(char_item_data, implicits = True)

{'BodyArmour': {'+X% to all Elemental Resistances': 1,
  '+X% to maximum Cold Resistance': 1,
  'Anger has X% increased Aura Effect': 1,
  'X% increased effect of Non-Curse Auras from your Skills': 1},
 'Gloves': {'Inflict Cold Exposure on Hit, applying -X% to Cold Resistance': 1,
  'Gain X Rage on Hit with Attacks, no more than once every X.X seconds': 1,
  'Projectiles Pierce an additional Target': 1,
  'Strike Skills target X additional nearby Enemy': 1},
 'Boots': {'X% chance to Avoid Elemental Ailments': 2,
  'Drops Brittle Ground while moving, lasting X seconds': 2},
 'Helm': {'Adds X to X Cold Damage': 1,
  'Your Hits treat Cold Resistance as X% higher than actual value': 1}}

In [65]:
def tally_ascendancy_skill_mods(accounts): 
    accounts = [(account[0], account[1]) for account in accounts]
    char_item_data = get_chars_item_data(accounts)
    item_tally = tally_item_mods(char_item_data)
    implicit_tally = tally_item_mods(char_item_data, implicits = True)
    return (item_tally, implicit_tally)

In [66]:
item_mods, implicit_mods = tally_ascendancy_skill_mods(accounts[0:5])

In [69]:
implicit_mods

{'BodyArmour': {'+X% to all Elemental Resistances': 2,
  'X% increased Armour': 1,
  'X% increased effect of Non-Curse Auras from your Skills': 2,
  'Pride has X% increased Aura Effect': 1},
 'Gloves': {'X% chance to Impale Enemies on Hit with Attacks': 2,
  'Gain X Rage on Hit with Attacks, no more than once every X.X seconds': 2,
  'While a Unique Enemy is in your Presence, X% chance to Intimidate Enemies for X seconds on Hit': 1,
  'While a Unique Enemy is in your Presence, Overwhelm X% Physical Damage Reduction': 1,
  'Overwhelm X% Physical Damage Reduction': 1,
  'X% chance to Intimidate Enemies for X seconds on Hit': 1},
 'Helm': {"Enemies you've Hit Recently have X% reduced Life Regeneration rate": 2,
  'X% increased Temporal Chains Curse Effect': 1,
  '+X% to maximum Lightning Resistance': 1,
  'X% increased Mana Reservation Efficiency of Skills': 1,
  'X% increased Damage per Power Charge': 1},
 'Boots': {'+X% to Fire and Cold Resistances': 1,
  'X% chance to Avoid being Poiso

In [47]:
mods = []
mod_tally = []
bases = []
for base_type in item_mods:
    mods+= (list(item_mods[base_type].keys()))
    mod_tally +=(list(item_mods[base_type].values()))
    bases +=([base_type for x in range(len(item_mods[base_type]))])

In [48]:
data = {'bases': bases, 'mods': mods, 'tally': mod_tally}
df = pd.DataFrame(data = data)


In [49]:
df.groupby('bases')

In [50]:
df

,bases,mods,tally
0,BodyArmour,Socketed Attacks have -X to Total Mana Cost,2
1,BodyArmour,X% increased maximum Life,4
2,BodyArmour,X% increased maximum Mana,4
3,BodyArmour,Recover X% of Life on Kill,1
4,BodyArmour,You can apply an additional Curse,1
...,...,...,...
179,Boots,+X% to Cold Resistance,2
180,Boots,X% increased Stun and Block Recovery,3
181,Boots,X% increased Armour,1
182,Boots,+X% to Lightning and Chaos Resistances,1


In [19]:
grouped_df = df.groupby('bases')
for key, item in grouped_df:
    print(grouped_df.get_group(key), "\n\n")

     bases                                               mods  tally
54  Amulet  Socketed Support Gems can also Support Skills ...      1
55  Amulet                             +X% to Fire Resistance      1
56  Amulet                             +X% to Cold Resistance      2
57  Amulet                        +X% to Lightning Resistance      2
58  Amulet                            -X% to Chaos Resistance      1
59  Amulet                                 +X to maximum Life      3
60  Amulet                                 +X to maximum Mana      1
61  Amulet                 X% increased Rarity of Items found      1
62  Amulet                            +X% to Chaos Resistance      1
63  Amulet      Channelling Skills have -X to Total Mana Cost      1
64  Amulet                                    +X to Dexterity      1
65  Amulet         X% increased Global Critical Strike Chance      1
66  Amulet                                X% increased Armour      1
67  Amulet                        